In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
#erreur sans ce package /opt/conda/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
!pip install langdetect


In [ ]:
import pandas as pd
import requests
import time
from urllib.parse import quote
from langdetect import detect
from fuzzywuzzy import fuzz 


In [ ]:
# List of French Writers abritrarily defined and chosen in the 17th, 18th and 19th century
authors = [
    # 17th century
    "Honoré d'Urfé", "Madeleine de Scudéry", "Paul Scarron", "Jean de La Fontaine",
    "Madame de Lafayette", "Charles Sorel", "Tristan L'Hermite", "François de Salignac de La Mothe-Fénelon",
    "Savinien de Cyrano de Bergerac",
    
    # 18th century
    "Montesquieu", "Voltaire", "Jean-Jacques Rousseau", "Denis Diderot", "Marivaux",
    "Abbé Prévost", "Pierre Choderlos de Laclos", "Beaumarchais", 
    
    # 19th century
    "Honoré de Balzac", "Victor Hugo", "Alexandre Dumas", "Gustave Flaubert", "Émile Zola",
    "Stendhal", "Alfred de Musset", "George Sand", "Jules Verne", "Alphonse Daudet",
    "Théophile Gautier", "Edmond de Goncourt",
    "Joris-Karl Huysmans", "Octave Mirbeau", 
    "Prosper Mérimée", "Eugène Sue", "Charles Nodier",
    "Gaston Leroux", "François-René de Chateaubriand", "Anatole France", "Gustave Flaubert", "Alfred Jarry",
    "Guy de Maupassant", "Romain Rolland", "Alfred Séguin", "Alfred de Vigny", "Paul de Kock"

]

# Create a DataFrame
df = pd.DataFrame(authors, columns=["Authors"])

df["Period"] = (
    ["17th century"] * 9 +
    ["18th century"] * 8 +
    ["19th century"] * 27
)

birth_years = [
    # 17th century
    1567, 1607, 1610, 1621, 1634, 1582, 1601, 1651, 1619,
    
    # 18th century
    1689, 1694, 1712, 1713, 1688, 1697, 1741, 1732,
    
    # 19th century
    1799, 1802, 1802, 1821, 1840, 1783, 1810, 1804, 
    1828, 1840, 1811, 1822, 1848, 1847, 1803, 1804, 
    1780, 1868, 1768, 1844, 1855, 1850, 1797, 1797, 
    1793, 1793, 1797
]

death_years = [
    # 17th century
    1625, 1701, 1660, 1695, 1693, 1654, 1655, 1715, 1655,
    
    # 18th century
    1755, 1778, 1778, 1784, 1763, 1763, 1803, 1799,
    
    # 19th century
    1850, 1885, 1870, 1880, 1902, 1842, 1857, 1876, 
    1905, 1897, 1872, 1896, 1907, 1917, 1870, 1857, 
    1844, 1927, 1848, 1924, 1887, 1916, 1863, 1863, 
    1870, 1892, 1863
]

# Add columns to the df
df["Birth"] = birth_years
df["Death"] = death_years

df


,Authors,Period,Birth,Death
0,Honoré d'Urfé,17th century,1567,1625
1,Madeleine de Scudéry,17th century,1607,1701
2,Paul Scarron,17th century,1610,1660
3,Jean de La Fontaine,17th century,1621,1695
4,Madame de Lafayette,17th century,1634,1693
5,Charles Sorel,17th century,1582,1654
6,Tristan L'Hermite,17th century,1601,1655
7,François de Salignac de La Mothe-Fénelon,17th century,1651,1715
8,Savinien de Cyrano de Bergerac,17th century,1619,1655
9,Montesquieu,18th century,1689,1755


In [ ]:
url_api = "https://openlibrary.org/search.json?"

# Function to get the title of the books and the themes
def get_random_book(author):
    url  = url_api + f'q=author:{author}' + '&fields=title,first_publish_year,subject'
    req = requests.get(url)
    time.sleep(2)
    # Check if the request worked
    if req.status_code == 200:
        books = req.json().get("docs", [])
        index = df[df["Authors"] == author].index[0] 
        
        books_with_themes = [
            book for book in books
            if book.get("subject")  # Check if subjects are defined
            and len(book["subject"]) >= 5  
            and book.get("first_publish_year")  # Check if the release date is available 
            and isinstance(book["first_publish_year"], int)  # Check if the release date is an int
            and df["Birth"][index] <= book["first_publish_year"]  # Check if the book was published after the birth of the author to avoid problems of namesake
            and book["first_publish_year"] <= df["Death"][index]  # Check if the book was published before death
        ] 
        num_books = len(books_with_themes)
        if num_books > 0:
            # We select the maximum number of books after filtering
            selected_books = books_with_themes
            book_info = []
            for info in selected_books:
                title = info.get("title", "N/A")
                themes = info.get("subject", "N/A")
                year = info.get("first_publish_year", "N/A")
                book_info.append((author, title, year, themes))
            return book_info
        else:
            return [(author, '', '','')]
    else:
        return [(author, '', '','')]
    


In [ ]:
list_books = []
for author in df["Authors"]:
    list_books.append(get_random_book(author))


[]
0

[{'first_publish_year': 1691, 'title': 'Artamène', 'subject': ['Court and courtiers', 'Fiction', 'Social conditions', 'Social life and customs', 'Women', 'Fiction, general', 'France, fiction']}]
1

[{'first_publish_year': 1649, 'title': 'Svr la conference de Rvel en mars', 'subject': ['Fronde', 'Poetry', 'History', 'France. 1649 March 11', 'France']}]
1

[{'first_publish_year': 1678, 'title': 'Fables', 'subject': ['French Fables', 'Translations into English', 'Fables', 'Oversize books', 'Illustrations', 'Adaptations', 'Translations into Malagasy', 'Specimens', "Aesop's fables", 'English Fables', 'French language', 'Translations into French Creole', 'Readers', 'Poetry', 'French language materials', 'Translations into Yiddish', 'Juvenile literature', 'Translations from French', 'Translations into Arabic', 'Translations into Esperanto', 'Translations into Occitan', 'Toy and movable books', 'Dialects', 'Latin language', 'Yiddish Fables', 'Translations into Italian', 'Translations', '

In [47]:
# As we have a : list[list[tuples]], we convert it to list[tuples] to transform it to a df afterward
flattened_books = []
for author_data in list_books:
    for book in author_data:
        flattened_books.append(book)

# Convertir en DataFrame
df_books = pd.DataFrame(flattened_books, columns=['Author', 'Title', 'Year', 'Themes'])


df_books


,Author,Title,Year,Themes
0,Honoré d'Urfé,,,
1,Madeleine de Scudéry,Artamène,1691,"[Court and courtiers, Fiction, Social conditio..."
2,Paul Scarron,Svr la conference de Rvel en mars,1649,"[Fronde, Poetry, History, France. 1649 March 1..."
3,Jean de La Fontaine,Fables,1678,"[French Fables, Translations into English, Fab..."
4,Jean de La Fontaine,Contes et nouvelles en vers,1685,"[Translations into English, Translations into ..."
...,...,...,...,...
188,Guy de Maupassant,,,
189,Romain Rolland,Musiciens d'aujourd'hui,1800,"[Music, History and criticism, Musicians, Biog..."
190,Alfred Séguin,,,
191,Alfred de Vigny,,,


In [48]:
# delete lines of the DF for which no book was found
df_books = df_books[df_books['Title'] != '']
df_books.reset_index(drop=True, inplace=True)


In [49]:
# Transform the list of themes in a string
df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(x))
df_books

/tmp/ipykernel_12013/1577876212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(x))


,Author,Title,Year,Themes
0,Madeleine de Scudéry,Artamène,1691,"Court and courtiers, Fiction, Social condition..."
1,Paul Scarron,Svr la conference de Rvel en mars,1649,"Fronde, Poetry, History, France. 1649 March 11..."
2,Jean de La Fontaine,Fables,1678,"French Fables, Translations into English, Fabl..."
3,Jean de La Fontaine,Contes et nouvelles en vers,1685,"Translations into English, Translations into O..."
4,François de Salignac de La Mothe-Fénelon,Aventures de Télémaque,1699,"Fiction, Telemachus (Greek mythology), Princes..."
...,...,...,...,...
175,Anatole France,The crime of Sylvestre Bonnard (member of the ...,1890,"Philologists, Men, Book collectors, Children a..."
176,Gustave Flaubert,Madame Bovary,1857,"literary fiction, French literature, Fiction, ..."
177,Gustave Flaubert,Salammbô,1863,"Fiction, History, Continental european fiction..."
178,Gustave Flaubert,Trois contes,1877,"Classic Literature, Fiction, French language, ..."


In [ ]:
url = "https://lingva.ml/api/v1/"
source = 'auto/' # auto as some themes may be written in another language than English
dest = 'fr/'
for i in range(len(df_books['Title'])):
    if detect(df_books['Title'][i]) != 'fr':
        url_api_transl = url + source + dest + quote(df_books['Title'][i])
        response = requests.get(url_api_transl)
        translated_themes = response.json().get("translation", "")
        df_books.loc[i,'Title'] = translated_themes
        time.sleep(2) # The API limit the flow of request


In [ ]:
def remove_similar_books_from_df(df, threshold=80):
    to_remove = []  # Liste pour enregistrer les indices des livres à supprimer

    # Grouper les livres par auteur
    list_books_per_author = df.groupby('Author')['Title'].apply(list)

    # Parcourir chaque groupe d'auteur
    for author, books in list_books_per_author.items():
        # Comparer tous les titres dans chaque groupe
        for i in range(len(books)):
            for j in range(i + 1, len(books)):  # Comparer un titre avec tous les autres après lui
                similarity = fuzz.ratio(books[i], books[j])
                if similarity >= threshold:
                    # Si les titres sont similaires, ajouter l'indice à supprimer
                    index_to_remove = df[(df['Author'] == author) & (df['Title'] == books[j])].index
                    to_remove.extend(index_to_remove)  # Ajouter l'indice à la liste

    # Supprimer les lignes contenant les indices marqués
    df = df.drop(to_remove).reset_index(drop=True)
    
    return df

In [56]:
df_books = remove_similar_books_from_df(df_books)

In [59]:
# The goal is now to clean the themes to only keep relevant themes, and to translate them into French
# To translate themes, we use the API Lingva

url = "https://lingva.ml/api/v1/"
source = 'auto/' # auto as some themes may be written in another language than English
dest = 'fr/'

for i in range(len(df_books['Themes'])):
    if detect(df_books['Themes'][i]) != 'fr' :
        url_api_transl = url + source + dest + df_books['Themes'][i]
        response = requests.get(url_api_transl)
        if response.status_code == 200:
            translated_themes = response.json().get("translation", "")
            df_books.loc[i,'Themes'] = translated_themes
        time.sleep(2)

In [64]:
#Map themes in order to clean themes in the df 
themes_mapping = {
    "Ambitions": ["Ambitions", "Luxe", "Richesse", "Pouvoir"],
    "Argent": ["Argent", "Finance", "Luxe", "Richesse"],
    "Arts et Culture": ["Musique", "Opéras", "Livret", "Partitions", "Illustrations"],
    "Biographie": ["Biographie", "Autobiographie", "Fiction biographique"],
    "Censure": ["Censure", "Critique littéraire", "Liberté d'expression", "Interdictions"],
    "Critique": ["Critique et interprétation", "Histoire et critique", "Critique", "Philosophie", "controverse", "Satire", "Histoire de la philosophie", "Histoire politique"],
    "Critique d'art": ["Critique d'art", "Art moderne", "Peinture", "Critique d'art", "Arts modernes"],
    "Coutumes": ["Coutumes", "Mœurs et coutumes"],
    "Décadence": ["Décadence", "Chute", "Esthétique", "Fiction"],
    "Drame": ["Drame", "Fiction", "Théâtre"],
    "Éducation": ["Éducation", "Éducation", "Ouvrages de jeunesse jusqu'à 1800"],
    "Éducation et Conduite de la vie": ["Éducation des filles", "Éducation des princes", "Conduite personnelle", "Éducation", "Philosophie de l'éducation"],
    "Épouvante": ["Épouvante", "Horreur", "Suspense", "Thriller"],
    "Esthétique": ["Esthétique", "Art"],
    "Fantastique": ["Fantastique", "Fiction fantastique", "Imaginaire"],
    "Fiction historique": ["Fiction historique", "Fiction", "Histoire", "Aventures", "Époque", "Événements"],
    "Héroïsme": ["Fiction", "Héroïsme", "Héroïsme"],
    "Histoire": ["Histoire", "Historique", "Historique et critique", "Société", "Évolutions sociales", "Révolution française", "Histoire", "Politique", "Révolutions"],
    "Inégalités": ["Inégalités", "Sociale"],
    "Instincts": ["Instincts", "Comportements"],
    "Intrigues": ["Fiction", "Intrigues"],
    "Justice": ["Justice", "Loi", "Droits", "Éthique"],
    "Liberté": ["Liberté", "Égalité", "Liberté religieuse", "Liberté"],
    "Luxe et débauche": ["Luxe et débauche", "Fiction", "Vie de cour", "Aristocratie", "Luxe", "Critique sociale", "Roi", "Reine"],
    "Mélancolie": ["Mélancolie", "Tristesse"],
    "Mœurs": ["Mœurs et coutumes", "Vie sociale et mœurs", "Mœurs et usages"],
    "Mythes et légendes": ["Mythes", "Mythologie", "Légendes", "Fables", "Folklore"],
    "Nature": ["Nature", "Histoire naturelle", "La nature dans la littérature", "Aspects religieux de la nature"],
    "Nature et environnement": ["Nature", "Écologie", "Environnement", "Conservation", "Vie à la campagne"],
    "Oppression": ["Oppression", "Répression"],
    "Passion": ["Passion", "Émotions", "Vie sociale et coutumes"],
    "Philosophie": ["Philosophie politique", "État", "Droit", "Liberté", "Nationalisme", "République", "Constitution", "Philosophie", "Philosophie du XVIIIe siècle", "Rationalisme", "Égalité", "Droit"],
    "Pouvoir": ["Pouvoir", "Droit", "Politique et gouvernement"],
    "Réalité et illusion": ["Réalité", "Illusion", "Perception", "L'irréel"],
    "Religion": ["Religion", "Christianisme", "Religieuses", "Jésuites", "Histoire de l'Église", "Église catholique", "Œuvres apologétiques", "Tolérance religieuse", "Affaire Calas", "Athéisme"],
    "Récits de guerre": ["Récits de guerre", "Conflits", "Histoire militaire"],
    "Roman": ["Roman", "Fiction", "Roman historique"],
    "Société et politique": ["Société", "Conditions sociales", "Structure sociale", "Mœurs", "Civilisation", "Culture", "Vie sociale", "État", "Science politique", "Éducation des princes", "Conduite de la vie", "Nationalisme", "Tolérance religieuse", "Conditions sociales", "Affaire","Réformateurs sociaux", "Socialisme"],
    "Théâtre": ["Théâtre", "Drame", "Théâtre en vers"],
    "Techniques": ["Technologie", "Science", "Innovations"],
    "Utopie": ["Utopie", "Littérature utopique", "Fiction spéculative", "Socialisme utopique"],
    "Voyages": ["Voyages", "Voyages imaginaires", "Voyages, imaginary", "Voyages"],
    "Vie sociale et mœurs": ["Vie sociale", "Mœurs et coutumes", "Conditions morales", "Civilisation"],
    "Voyages et explorations": ["Voyages", "Voyages et découvertes", "Explorations"],
    "Poésie": ["Poésie", "Vers", "Chansons", "Épique", "Lyrique", "Sonnet", "Ballade"],
    "Romans": ["Romans", "Roman historique", "Roman d'aventure", "Roman philosophique", "Roman social", "Roman d'amour"],
    "Fables": ["Fables", "Morales", "Allégories", "Contes moraux", "Animaux dans la littérature"],
    "Satire": ["Satire", "Humour", "Critique sociale", "Ironie", "Caricature", "Sarcasme"],
    "Contes": ["Contes", "Fiction pour enfants", "Féerie", "Contes merveilleux", "Fables", "Contes populaires"],
    "Lettres imaginaires": ["Lettres imaginaires", "Correspondance fictive", "Épistolaire", "Narration par lettres", "Philosophie épistolaire"],
    "Voyages": ["Voyages", "Voyages imaginaires", "Explorations", "Voyages réels", "Aventures géographiques", "Découvertes", "Voyages philosophiques", "Aventure et aventuriers", "Histoires d'aventure", "action et aventure"],
}



In [65]:
# Fonction de mapping des thèmes
def map_themes(row):
    # Diviser les thèmes par virgule
    themes = row.split(", ")
    mapped_themes = []
    
    # Vérifier et mapper chaque thème
    for theme in themes:
        # Chercher le thème dans le dictionnaire
        for key, values in themes_mapping.items():
            if theme in values:
                mapped_themes.append(key)
                break
    
    # Retourner les thèmes mappés sous forme de chaîne
    return ", ".join(mapped_themes)

# Appliquer la fonction de mapping sur la colonne "Themes"
df_books["Themes"] = df_books["Themes"].apply(map_themes)


In [67]:
# Remove empty lines of the df as iit means that themes obtained in the API where not relevant for our analysis
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)


In [68]:
df_books.reset_index(drop=True, inplace=True)

In [82]:
# Remove duplicated themes
df_books['Themes'] = df_books['Themes'].apply(lambda x: set(x.split(", ")))


In [84]:
# Transform the list of themes in a string
df_books['Themes'] = df_books['Themes'].apply(lambda x: ", ".join(x))

In [87]:
df_books.to_csv('books.csv', index=False)

In [ ]:
df_books = pd.read_csv('Data/books.csv')
